In [1]:
# Célula 1 Montar o Google Drive e acessar o dataset combined_full.json na pasta datasets

from google.colab import drive
drive.mount('/content/drive')

# Verificar se o arquivo está acessível
!ls /content/drive/MyDrive/datasets/combined_full.json


Mounted at /content/drive
/content/drive/MyDrive/datasets/combined_full.json


In [2]:
# NOVA CÉLULA 2: Definições e Funções de Checkpoint

import os
import torch

# Diretório para salvar checkpoints no Google Drive
checkpoint_dir = "/content/drive/MyDrive/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pth")
print(f"Caminho do Checkpoint: {checkpoint_path}")


# Função para salvar checkpoint (SEU CÓDIGO ORIGINAL)
def save_checkpoint(epoch, model, optimizer, loss, accuracy):
    state = {
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'loss': loss,
        'accuracy': accuracy
    }
    # Ajuste: Salvar com o número da época no print para clareza
    torch.save(state, checkpoint_path)
    print(f"Checkpoint salvo, finalizando época {epoch + 1}.")

# Função para carregar checkpoint (SEU CÓDIGO ORIGINAL COM AJUSTE)
def load_checkpoint(model, optimizer):
    if os.path.isfile(checkpoint_path):
        # AJUSTE DE MAPA DE LOCALIZAÇÃO: Permite carregar mesmo se a GPU for trocada
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        # Retoma da PRÓXIMA época
        start_epoch = checkpoint['epoch'] + 1
        print(f"Checkpoint carregado, retomando da ÉPOCA {start_epoch}.")
        return start_epoch
    else:
        print("Nenhum checkpoint encontrado, iniciando do zero (ÉPOCA 1).")
        return 0

# Função principal para treino com checkpoints (SEU CÓDIGO ORIGINAL)
def train(num_epochs, model, optimizer, criterion, train_loader, val_loader=None, start_epoch=0, checkpoint_interval=1):
    # Seu código de loop de treinamento aqui
    # ...
    pass

Caminho do Checkpoint: /content/drive/MyDrive/checkpoints/checkpoint.pth


In [3]:
# Célula 3 - Carregar o dataset combined_full.json do Google Drive e importar bibliotecas para pré-processamento

import json
import pandas as pd

# Caminho completo do arquivo no Google Drive
dataset_path = '/content/drive/MyDrive/datasets/combined_full.json'

# Abrir e carregar o arquivo JSON
with open(dataset_path, 'r') as file:
    data = json.load(file)

# Exemplo: converter JSON para DataFrame para manipulação fácil
df = pd.json_normalize(data)

# Mostrar as primeiras linhas para conferir o carregamento
print(df.head())


                                                text  label
0  http://applevalleyhyundai.org/slug/c/casio-ctk...      0
1  hannibal.net/news_local/x342379827/Illinois-Ri...      0
2        linkedin.com/company/taipei-american-school      0
3                               http://wwwgoogle.ro/      1
4                 www.ganges.com/rss/videos_top.html      0


In [4]:
# Célula 4 - Pré-processar o dataset: limpar URLs, preparar colunas para treinamento e separar atributos da label

# Exibir as colunas para confirmação
print(df.columns)

# Limpar URLs/textos na coluna 'text' removendo prefixos http(s) e www
df['text'] = df['text'].str.replace(r'http[s]?://', '', regex=True)
df['text'] = df['text'].str.replace(r'www\.', '', regex=True)

# Separar features (coluna text) e label (coluna label)
X = df['text']
y = df['label']

# Conferir as primeiras linhas para validação
print(X.head())
print(y.head())



Index(['text', 'label'], dtype='object')
0    applevalleyhyundai.org/slug/c/casio-ctk-630-el...
1    hannibal.net/news_local/x342379827/Illinois-Ri...
2          linkedin.com/company/taipei-american-school
3                                        wwwgoogle.ro/
4                       ganges.com/rss/videos_top.html
Name: text, dtype: object
0    0
1    0
2    0
3    1
4    0
Name: label, dtype: int64


In [5]:
# Célula 5: Pré-processamento - Tokenização e Limpeza dos Dados
# Esta célula realiza as etapas de pré-processamento essenciais para preparar os dados textuais
# para o modelo de deep learning. Inclui limpeza básica, tokenização, remoção de stop words,
# e outras transformações necessárias para melhorar o desempenho do modelo na tarefa de phishing.

# Tokenização, Split e Criação de DataLoaders para DistilBERT

from transformers import DistilBertTokenizerFast
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

# 1. Carregar o Tokenizer do DistilBERT (Ele é quem faz a 'limpeza' e tokenização)
# Você já definiu X (features) e y (labels) nas células anteriores
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# 2. Dividir os dados em conjuntos de treino e validação
# X: série de textos (já limpa de URLs na Célula 4)
# y: série de labels
train_texts, val_texts, train_labels, val_labels = train_test_split(
    X.tolist(), # Convertendo para lista
    y.tolist(), # Convertendo para lista
    test_size=0.2, # Exemplo de 20% para validação
    random_state=42,
    stratify=y.tolist() # Importante para balanceamento de classes (phishing/não-phishing)
)

# 3. Tokenização Usando o Tokenizer do DistilBERT
# max_length depende do tamanho médio/máximo dos seus textos
max_len = 128

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_len)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_len)

# 4. Criar um Objeto Dataset Customizado (Padrão para PyTorch/Transformers)
class PhishingDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Transforma os encodings do tokenizer em tensores PyTorch
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 5. Instanciar DataSets e DataLoaders (CRIAÇÃO DAS VARIÁVEIS CHAVE)
train_dataset = PhishingDataset(train_encodings, train_labels)
val_dataset = PhishingDataset(val_encodings, val_labels)

BATCH_SIZE = 16 # Ajuste conforme a memória da sua GPU (T4/V100)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print("Dados pré-processados, DataLoaders prontos.")
# Agora temos as variáveis globais: train_loader e val_loader


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Dados pré-processados, DataLoaders prontos.


In [ ]:
# NOVA CÉLULA 6: Setup do Modelo e Otimizador (Substitui sua antiga Célula 4 de treino). Setup do Modelo (DistilBERT) e Funções Auxiliares

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

# 1. Definir o Device e Variáveis de Dados (Assume que 'df' existe da Célula 3/4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
X = df['text'] # Features carregadas da Célula 4
y = df['label'] # Labels carregadas da Célula 4

# 2. Definição da Classe PhishingDataset (Seu código original)
class PhishingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 3. Inicialização do Modelo, Tokenizer e DataLoaders
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

# Split de Dados e Criação de Datasets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    X, y, test_size=0.2, random_state=42
)
train_dataset = PhishingDataset(train_texts, train_labels, tokenizer)
val_dataset = PhishingDataset(val_texts, val_labels, tokenizer)

# Criação de DataLoaders
BATCH_SIZE = 16 # Mantendo seu ajuste
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Otimizador e Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 20 # Mantemos o máximo
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)


# 4. FUNÇÕES DE TREINAMENTO E AVALIAÇÃO (Seu código original)

def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(data_loader), correct / total

def train_epoch_verbose(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for i, batch in enumerate(tqdm(data_loader, desc='Treinando batches')):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        if (i + 1) % 10 == 0:
            print(f"Batch {i+1}, Loss parcial: {total_loss/(i+1):.4f}, Acurácia parcial: {correct/total*100:.2f}%")

    return total_loss / len(data_loader), correct / total


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# NOVA CÉLULA 7: Lógica de Reinício e Execução do Treinamento. Loop Principal de Treinamento
# Ela isola o controle de fluxo, carrega o progresso anterior (se existir) e inicia o loop de treinamento.
# 1. FUNÇÕES DE CHECKPOINT (Ajustadas para usar o caminho do Drive)
# Necessário redefinir aqui, ou garantir que a Célula 2 as exportou globalmente
# Usaremos o checkpoint_path definido na Célula 2.

def save_checkpoint(state, filename):
    # filename deve ser o 'checkpoint_path' completo do Drive
    torch.save(state, filename)
    print(f"Checkpoint salvo em {filename}")

def load_checkpoint(model, optimizer, scheduler, filename):
    # filename deve ser o 'checkpoint_path' completo do Drive
    if os.path.isfile(filename):
        # AJUSTE: map_location garante que funcione em qualquer GPU
        checkpoint = torch.load(filename, map_location=device)

        # Carregamento dos estados
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        start_epoch = checkpoint['epoch'] + 1
        print(f"Checkpoint carregado, retomando da época {start_epoch}")
        return start_epoch
    else:
        print("Nenhum checkpoint encontrado, começando da Época 1")
        return 0

# 2. INICIAR A LÓGICA DE REINÍCIO

history = { # Re-defina o histórico para garantir que exista
    'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []
}
patience = 3
best_val_loss = np.inf
epochs_no_improve = 0

# AQUI: Chamada crucial que define o ponto de partida!
# 'checkpoint_path' deve estar definido na Célula 2
start_epoch = load_checkpoint(model, optimizer, scheduler, checkpoint_path)
print(f"Treinamento irá começar na época: {start_epoch + 1}")


# 3. LOOP DE TREINAMENTO PRINCIPAL (Seu código original)

for epoch in range(start_epoch, num_epochs):
    if device.type == 'cuda':
        torch.cuda.empty_cache()

    print(f"\nIniciando época {epoch+1}/{num_epochs}")
    train_loss, train_acc = train_epoch_verbose(model, train_loader, optimizer, scheduler, device)
    val_loss, val_acc = evaluate(model, val_loader, device)
    print(f"Loss de treino: {train_loss:.4f} | Acurácia de treino: {train_acc*100:.2f}%")
    print(f"Loss de validação: {val_loss:.4f} | Acurácia de validação: {val_acc*100:.2f}%")

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)

    # SALVAR CHECKPOINT
    save_checkpoint({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
    }, checkpoint_path) # Usa o caminho COMPLETO do Drive

    # Lógica de Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping acionado. Parando treinamento.")
            break


Nenhum checkpoint encontrado, começando da Época 1
Treinamento irá começar na época: 1

Iniciando época 1/20


Treinando batches:   0%|          | 11/43580 [00:03<2:07:29,  5.70it/s]

Batch 10, Loss parcial: 0.6791, Acurácia parcial: 50.00%


Treinando batches:   0%|          | 21/43580 [00:04<1:55:17,  6.30it/s]

Batch 20, Loss parcial: 0.6411, Acurácia parcial: 61.56%


...

Treinando batches: 100%|█████████▉| 43570/43580 [2:11:36<00:01,  5.64it/s]

Batch 43570, Loss parcial: 0.0734, Acurácia parcial: 97.32%


Treinando batches: 100%|██████████| 43580/43580 [2:11:38<00:00,  5.52it/s]


Batch 43580, Loss parcial: 0.0734, Acurácia parcial: 97.32%


KeyboardInterrupt: 

In [ ]:
# NOVA CÉLULA 8: Análise e Visualização dos Resultados do Modelo. Plotagem

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_curve, auc
import seaborn as sns
import torch
import numpy as np
# Não precisamos importar o DataLoader se ele já foi importado na Célula 5.

# 1. Função para avaliar o modelo e gerar os gráficos de Matriz de Confusão e Curva ROC
def avaliar_modelo(model, dataloader, device):
    """Calcula métricas de desempenho e plota a Matriz de Confusão e a Curva ROC."""
    # ... (Seu código completo da função avaliar_modelo) ...
    # Mantenha o corpo da sua função original aqui.
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    # Nome das classes para os gráficos
    classes = ['Não Phishing (0)', 'Phishing (1)']

    print("--- Avaliando o Modelo ---")
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=inputs, attention_mask=attention_mask)
            logits = outputs.logits

            # Como o DistilBERTForSequenceClassification retorna logits, usamos softmax/argmax
            if logits.shape[1] > 1:
                probs = torch.softmax(logits, dim=1)[:, 1]
                preds = torch.argmax(logits, dim=1)
            else:
                probs = torch.sigmoid(logits).squeeze()
                preds = (probs > 0.5).long()

            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # --- Cálculo das Métricas (Mantenha seu código original) ---
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)

    # --- Impressão dos Resultados (Mantenha seu código original) ---
    print(f'Acurácia: {acc * 100:.2f}%')
    # ... (demais prints) ...

    # --- Plotagens (Mantenha seu código original) ---
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes,
                yticklabels=classes)
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()
    print(f'VP: {cm[1, 1]} | VN: {cm[0, 0]} | FP: {cm[0, 1]} | FN: {cm[1, 0]}')

    plt.figure(figsize=(7, 5))
    plt.plot(fpr, tpr, color='darkorange', label=f'Curva ROC (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlabel('Taxa de Falso Positivo (FPR)')
    plt.ylabel('Taxa de Verdadeiro Positivo (TPR)')
    plt.title('Curva ROC')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()


# 2. Função para plotar as curvas de Loss e Acurácia (Histórico de Treinamento)
def plotar_curvas(history):
    # ... (Seu código completo da função plotar_curvas) ...
    # Mantenha o corpo da sua função original aqui.
    print("\n--- Plotando Curvas de Treinamento ---")
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))

    # Curvas de Loss (Mantenha seu código original)
    if 'train_loss' in history and 'val_loss' in history:
        ax[0].plot(history['train_loss'], label='Loss Treino')
        ax[0].plot(history['val_loss'], label='Loss Validação')
        ax[0].set_title('Curvas de Loss')
        ax[0].set_xlabel('Épocas')
        ax[0].set_ylabel('Loss')
        ax[0].legend()
    else:
        ax[0].set_title('Dados de Loss Indisponíveis')

    # Curvas de Acurácia (Mantenha seu código original)
    if 'train_acc' in history and 'val_acc' in history:
        ax[1].plot(history['train_acc'], label='Acurácia Treino')
        ax[1].plot(history['val_acc'], label='Acurácia Validação')
        ax[1].set_title('Curvas de Acurácia')
        ax[1].set_xlabel('Épocas')
        ax[1].set_ylabel('Acurácia')
        ax[1].legend()
    else:
        ax[1].set_title('Dados de Acurácia Indisponíveis')

    plt.tight_layout()
    plt.show()


# 3. CHAMADA DE EXECUÇÃO (USANDO VARIÁVEIS GLOBAIS DEFINIDAS NAS CÉLULAS 5 E 6)
try:
    # 1. Avalia e plota Matriz de Confusão e Curva ROC
    avaliar_modelo(model, val_loader, device)

    # 2. Plota Curvas de Treinamento
    plotar_curvas(history)

except NameError as e:
    print(f"\nERRO: Variável não definida. Certifique-se de que 'model', 'val_loader', 'device' e 'history' foram definidos e as células anteriores (5 e 6) foram executadas.\nDetalhe do erro: {e}")
except Exception as e:
    print(f"\nOcorreu um erro ao avaliar ou plotar: {e}")


ERRO: Variável não definida. Certifique-se de que 'model', 'val_loader', 'device' e 'history' foram definidos e as células anteriores (5 e 6) foram executadas.
Detalhe do erro: name 'model' is not defined


In [8]:
# CÉLULA 10: Lógica de Reinício e Loop Principal de Treinamento Híbrido

# Importações de segurança (garantindo que tudo esteja carregado)
import torch
import numpy as np
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import os

# --- 1. CONFIGURAÇÃO DE TREINAMENTO ---

# Assumimos que 'model_hybrid' foi definido na Célula 9, 'train_loader' e 'val_loader' na Célula 5.

# Otimizador e Scheduler (Definidos para o NOVO modelo HÍBRIDO)
optimizer_hybrid = AdamW(model_hybrid.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs_hybrid = 20 # Mantemos o máximo
total_steps_hybrid = len(train_loader) * num_epochs_hybrid
scheduler_hybrid = get_linear_schedule_with_warmup(
    optimizer_hybrid, num_warmup_steps=0, num_training_steps=total_steps_hybrid
)

# AJUSTE NO CHECKPOINT: Usaremos um novo nome para não sobrescrever o DistilBERT
checkpoint_path_hybrid = "/content/drive/MyDrive/checkpoints/checkpoint_hybrid.pth"


# --- 2. FUNÇÕES DE CHECKPOINT HÍBRIDO ---
# Requeremos a redefinição aqui para garantir que o 'checkpoint_path_hybrid' seja usado corretamente.

def save_checkpoint_hybrid(state, filename):
    # Salva o checkpoint HÍBRIDO no Drive
    torch.save(state, filename)
    print(f"Checkpoint HÍBRIDO salvo em {filename}")

def load_checkpoint_hybrid(model, optimizer, scheduler, filename):
    # Carrega o checkpoint HÍBRIDO do Drive
    if os.path.isfile(filename):
        # map_location garante que o carregamento funcione independente da GPU
        checkpoint = torch.load(filename, map_location=device)

        # Carregamento dos estados
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        start_epoch = checkpoint['epoch'] + 1
        print(f"Checkpoint HÍBRIDO carregado, retomando da época {start_epoch}")
        return start_epoch
    else:
        print("Nenhum checkpoint HÍBRIDO encontrado, começando da Época 1")
        return 0

# --- 3. INICIAR A LÓGICA DE REINÍCIO E LOOP ---

history_hybrid = { # Novo histórico para o modelo híbrido
    'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []
}
patience = 3
best_val_loss = np.inf
epochs_no_improve = 0

# AQUI: Carrega o checkpoint HÍBRIDO para definir o ponto de partida
start_epoch_hybrid = load_checkpoint_hybrid(model_hybrid, optimizer_hybrid, scheduler_hybrid, checkpoint_path_hybrid)
print(f"Treinamento HÍBRIDO irá começar na época: {start_epoch_hybrid + 1}")


# LOOP DE TREINAMENTO PRINCIPAL
# As funções 'train_epoch_verbose' e 'evaluate' da Célula 5 serão utilizadas.

for epoch in range(start_epoch_hybrid, num_epochs_hybrid):
    if device.type == 'cuda':
        torch.cuda.empty_cache()

    print(f"\nIniciando época {epoch+1}/{num_epochs_hybrid} (Modelo Híbrido)")

    # Treino e Avaliação: Usamos as funções auxiliares da Célula 5/6
    train_loss, train_acc = train_epoch_verbose(model_hybrid, train_loader, optimizer_hybrid, scheduler_hybrid, device)
    val_loss, val_acc = evaluate(model_hybrid, val_loader, device)

    print(f"Loss de treino: {train_loss:.4f} | Acurácia de treino: {train_acc*100:.2f}%")
    print(f"Loss de validação: {val_loss:.4f} | Acurácia de validação: {val_acc*100:.2f}%")

    # Armazenamento do histórico
    history_hybrid['train_loss'].append(train_loss)
    history_hybrid['val_loss'].append(val_loss)
    history_hybrid['train_acc'].append(train_acc)
    history_hybrid['val_acc'].append(val_acc)

    # SALVAR CHECKPOINT HÍBRIDO
    save_checkpoint_hybrid({
        'epoch': epoch,
        'model_state_dict': model_hybrid.state_dict(),
        'optimizer_state_dict': optimizer_hybrid.state_dict(),
        'scheduler_state_dict': scheduler_hybrid.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
    }, checkpoint_path_hybrid)

    # Lógica de Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping acionado para o Modelo Híbrido. Parando treinamento.")
            break

Nenhum checkpoint HÍBRIDO encontrado, começando da Época 1
Treinamento HÍBRIDO irá começar na época: 1

Iniciando época 1/20 (Modelo Híbrido)


Treinando batches:   0%|          | 0/43580 [00:00<?, ?it/s]


KeyError: 'label'